# Xception Model without DataAugmentation

In [1]:
# Pre trained Xception weight 
# RGB Image 
# The original size of the image for the pretrained Xception model is 224x224 (224x224x3) 
# The size of our input image is resized to 128x128 (128x128x3)

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model

Using TensorFlow backend.


###### Loading data
Intially the datas are structured to input to the convolutional layer by converting the images in to numpy array, normalized (changing the range of pixel intensity values) and reshaped accordingly. Finally, stored in the path shown below.

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB_Sequence_128/train/x_data_all.npy')
y_train = np.load('/cai_lab/thomas/DWD/Numpy_RGB_Sequence_128/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB_Sequence_128/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/Numpy_RGB_Sequence_128/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (11878, 5, 128, 128, 3) y_train.shape: (11878, 9)
x_valid.shape: (1858, 5, 128, 128, 3) y_valid.shape: (1858, 9)


In [4]:
batch_size = 24
resize = 128     # The image is resized to 128
n_class = 9


## Xception Model

In [5]:
from keras.applications.xception import preprocess_input, decode_predictions
from keras.applications.xception import Xception

inputs = Input (shape = (resize, resize, 3))

xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (224,224,3))
xception_conv.summary()





W0511 03:18:48.476632 140122739902208 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



83689472/83683744 [==============================] - 23s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
_________________________

In [6]:

output_xception = xception_conv(inputs)

x=GlobalAveragePooling2D()(output_xception)


dense_layer = Dense(1024, activation = 'relu')(x)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(1024, activation = 'relu')(dense_layer)
dense_layer = BatchNormalization()(dense_layer)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(512, activation = 'relu')(dense_layer)


final_output = Dense(n_class, activation = 'softmax')(dense_layer)


final_model = Model ([inputs], final_output)

final_model.summary()



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 128, 128, 3)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 2048)           20861480  
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               2360320   
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_5 (Batch (None, 1024)              4096

### Model Saving and Compling 

In [7]:

save_name = 'Comp_Xception_RGB_128'
save_path = './checkpoints/' + save_name

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

final_model = multi_gpu_model(final_model, gpus=2)

# aug = ImageDataGenerator(
#         rotation_range=20,
#         zoom_range=0.15,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.15,
#         horizontal_flip=True,
#         fill_mode="nearest")


# Compiling
sgd = optimizers.SGD(lr = 0.001)

final_model.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])

    
final_model.fit(x_train, y_train, 
                     batch_size=batch_size,
                    validation_data=(x_valid, y_valid),       
                    epochs=60,
                    verbose=1,
                    callbacks=callback_list)

final_model.save(save_path + '/epoch-finish.h5')
final_model.save_weights(save_path +'/epoch-finish_weights.h5')

W0511 03:19:27.303833 140122739902208 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0511 03:19:37.936196 140122739902208 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0511 03:19:37.955370 140122739902208 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0511 03:19:37.956166 140122739902208 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Train on 11878 samples, validate on 1858 samples
Epoch 1/60
11878/11878 [==============================] - 331s 28ms/step - loss: 2.0912 - accuracy: 0.2394 - val_loss: 1.8551 - val_accuracy: 0.4290

Epoch 00001: saving model to ./checkpoints/RGB_128_Xception_and_LSTM/checkpoint-0001.h5


W0511 03:25:33.196466 140122739902208 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/60
11878/11878 [==============================] - 314s 26ms/step - loss: 1.3391 - accuracy: 0.5465 - val_loss: 1.4519 - val_accuracy: 0.4903

Epoch 00002: saving model to ./checkpoints/RGB_128_Xception_and_LSTM/checkpoint-0002.h5
Epoch 3/60
11878/11878 [==============================] - 317s 27ms/step - loss: 0.7016 - accuracy: 0.7825 - val_loss: 1.3467 - val_accuracy: 0.5484

Epoch 00003: saving model to ./checkpoints/RGB_128_Xception_and_LSTM/checkpoint-0003.h5
Epoch 4/60
11878/11878 [==============================] - 314s 26ms/step - loss: 0.3916 - accuracy: 0.8854 - val_loss: 1.4117 - val_accuracy: 0.5662

Epoch 00004: saving model to ./checkpoints/RGB_128_Xception_and_LSTM/checkpoint-0004.h5
Epoch 5/60
11878/11878 [==============================] - 317s 27ms/step - loss: 0.2511 - accuracy: 0.9290 - val_loss: 1.4383 - val_accuracy: 0.5797

Epoch 00005: saving model to ./checkpoints/RGB_128_Xception_and_LSTM/checkpoint-0005.h5
Epoch 6/60
11878/11878 [========================